In [ ]:
import numpy as np

实现其中核心的算法

In [ ]:

def softer_nms(dets, confidence=None, thresh=0.01, sigma=0.5, ax=None):

    N = len(dets)
    x1 = dets[:, 0].copy()
    y1 = dets[:, 1].copy()
    x2 = dets[:, 2].copy()
    y2 = dets[:, 3].copy()
    scores = dets[:, 4].copy()
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    ious = np.zeros((N, N))
    kls = np.zeros((N, N))
    for i in range(N):
        xx1 = np.maximum(x1[i], x1)
        yy1 = np.maximum(y1[i], y1)
        xx2 = np.minimum(x2[i], x2)
        yy2 = np.minimum(y2[i], y2)

        w = np.maximum(0.0, xx2 - xx1 + 1.)
        h = np.maximum(0.0, yy2 - yy1 + 1.)
        inter = w * h
        ovr = inter / (areas[i] + areas - inter)
        ious[i, :] = ovr.copy()

    i = 0
    while i < N:
        maxpos = dets[i:N, 4].argmax()
        maxpos += i
        dets[[maxpos, i]] = dets[[i, maxpos]]
        confidence[[maxpos, i]] = confidence[[i, maxpos]]
        ious[[maxpos, i]] = ious[[i, maxpos]]
        ious[:, [maxpos, i]] = ious[:, [i, maxpos]]

        ovr_bbox = np.where((ious[i, i:N] > thresh))[0] + i

        pos = i + 1
        while pos < N:
            if ious[i, pos] > 0:
                ovr = ious[i, pos]
                dets[pos, 4] *= np.exp(-(ovr * ovr) / sigma)
                if dets[pos, 4] < 0.001:
                    dets[[pos, N - 1]] = dets[[N - 1, pos]]
                    confidence[[pos, N - 1]] = confidence[[N - 1, pos]]
                    ious[[pos, N - 1]] = ious[[N - 1, pos]]
                    ious[:, [pos, N - 1]] = ious[:, [N - 1, pos]]
                    N -= 1
                    pos -= 1
            pos += 1
        i += 1
    keep = [i for i in range(N)]
    return dets[keep], keep

通过模拟的数据进行测试验证

In [ ]:

if __name__ == '__main__':
    # 创建测试数据
    test_boxes = np.array([
        [100, 100, 200, 200, 0.9],  # x1, y1, x2, y2, score
        [110, 110, 210, 210, 0.8],
        [120, 120, 220, 220, 0.7],
        [130, 130, 230, 230, 0.6],
        [200, 200, 300, 300, 0.95],
    ], dtype=np.float32)
    
    # 创建置信度数组（可选）
    confidence = np.array([0.9, 0.85, 0.75, 0.65, 0.95])
    
    # 设置参数
    thresh = 0.3      # IoU阈值
    sigma = 0.5       # 高斯函数的sigma参数
    
    print("原始检测框：")
    print("格式：[x1, y1, x2, y2, score]")
    for i, box in enumerate(test_boxes):
        print(f"Box {i+1}: {box}, confidence: {confidence[i]:.3f}")
    
    # 运行Softer-NMS
    filtered_boxes, keep_indices = softer_nms(
        dets=test_boxes,
        confidence=confidence,
        thresh=thresh,
        sigma=sigma
    )
    
    print("\nSofter-NMS 后的检测框：")
    print("格式：[x1, y1, x2, y2, score]")
    for i, box in enumerate(filtered_boxes):
        print(f"Box {i+1}: {box}, confidence: {confidence[keep_indices[i]]:.3f}")
    
    # 输出统计信息
    print("\n统计信息：")
    print(f"原始检测框数量: {len(test_boxes)}")
    print(f"过滤后检测框数量: {len(filtered_boxes)}")
    print(f"使用的IoU阈值: {thresh}")
    print(f"使用的sigma值: {sigma}")
    
    # 计算保留率
    retention_rate = len(filtered_boxes) / len(test_boxes) * 100
    print(f"检测框保留率: {retention_rate:.1f}%")